In [1]:
import ast
import jnc
import json
import httpx
import string
from fuzzywuzzy import fuzz
from num2words import num2words
from googletrans import Translator
from tqdm.autonotebook import tqdm
from nltk.stem import PorterStemmer

/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/home/phucpx/miniconda3/envs/phucpx/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  if __name__ == '__main__':


In [14]:
timeout = httpx.Timeout(30) 
gg_translator = Translator(timeout=timeout)

ps = PorterStemmer()

vi_num2words_dict = {
    0: 'không',
    1: 'một',
    2: 'hai',
    3: 'ba',
    4: 'bốn',
    5: 'năm',
    6: 'sáu',
    7: 'bảy',
    8: 'tám',
    9: 'chín',
    10: 'mười'
}
vi_words2num_dict = {
    'không': 0,
    'một': 1,
    'hai': 2,
    'ba': 3,
    'bốn': 4,
    'năm': 5,
    'sáu': 6,
    'bảy': 7,
    'tám': 8,
    'chín': 9,
    'mười': 10
}

en_words2num_dict = {
    'zero': 0,
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9,
    'ten': 10
}

no_ja_dict = {
    '一': 1,
    '二': 2,
    '三': 3,
    
}

list_numbers = [
    'không', 'một', 'hai', 'ba', 'bốn', 'năm', 'sáu', 'bảy', 'tám', 'chín', 'mười',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
    'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'a ', 'an '
]

def remove_punc(text):
    text = text.lower()
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)


def get_samples_by_id(image_id, annotations, question_type):
    return_samples = []
    for anno in annotations:
        if anno['image_id'] == image_id:
            if anno['question_type'] == question_type:
                return_samples.append(anno)

    return return_samples

def get_by_language(annotations, language='en'):
    tmp = []
    for anno in annotations:
        if anno['language'] == language:
            tmp.append(anno)

    return tmp

def get_idx_in_list(idx, list_triple):
    for triple in list_triple:
        if idx in triple:
            return triple
    
    return None

def check_token_number(token):
    try:
        flag = str(jnc.ja_to_arabic(token)).isdigit()
    except:
        flag = False

    if token in list_numbers or flag:
        return True
    
    return False

In [3]:
group_questions = []

with open('./data/test/evjvqa_public_test-lang-qtype-answer.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
    
test_annotations = test_data['annotations']


with open('./outputs/public-test/results-ofa-huge-finetuned.json', 'r', encoding='utf-8') as f:
    ofa = json.load(f)


with open('./outputs/public-test/results-vit-mt5.json', 'r', encoding='utf-8') as f:
    mt5vit = json.load(f)

In [4]:
with open('./data/test/group.md', 'r', encoding='utf-8') as f:
    group_ids = [ast.literal_eval(line.strip()) for line in f.readlines()]

In [5]:
group_ids[0], type(group_ids[0])

([0, 2, 4], list)

## HOW_MANY

In [19]:
answer_dict = {}

for i, anno in tqdm(enumerate(test_annotations)):
    question_type = anno['question_type']
    question = anno['question']
    
    if question_type == 'HOW_MANY':
        idx = anno['id']
        if anno['language'] == 'en':
            answer = ofa[str(idx)]
            if len(answer.split()) == 2:
                answer = answer.replace('the ', ' ').strip() 
            if '⁄' in answer: 
                tmp_ans = answer.split('⁄')
                ans = []
                for w in tmp_ans:
                    ans.append(num2words(int(w)).replace('-', ' '))
                
                answer = ' '.join(ans)

            answer_dict[idx] = answer
        
        elif anno['language'] == 'vi':
            similar_ids = get_idx_in_list(idx, group_ids)[0]
            
            answer = mt5vit[str(idx)]
            
            if similar_ids is not None:
                en_answer = answer_dict[similar_ids]
                vi_answer = gg_translator.translate(en_answer, src='en', dest=anno['language']).text
                vi_answer = vi_answer.replace('số', ' ').strip()
                
                if 'bao nhiêu' in question:
                    if "năm" in answer and len(vi_answer.split()) == 1:
                        tmp_answer = answer.replace('năm', vi_answer)
                    else:
                        tmp_answer = question.replace("?", '').replace("bao nhiêu", vi_answer)
            
#                 tokens = answer.split()
#                 for j in range(len(tokens)):
#                     if tokens[j] in list_numbers:
#                         tokens[j] = vi_answer

#                 answer =  ' '.join(tokens)
                answer = tmp_answer
            
            answer_dict[idx] = answer
        
        elif anno['language'] == 'ja':
            similar_ids = get_idx_in_list(idx, group_ids)[0]
            
            answer = mt5vit[str(idx)]
            
            answer_dict[idx] = answer
            
            if similar_ids is not None:
                en_answer = answer_dict[similar_ids]
                try:
                    digit_answer = en_words2num_dict[en_answer]

                    tokens = list(answer)
                    for j in range(len(tokens)):
                        if check_token_number(str(tokens[j])):
                            tokens[j] = str(digit_answer)

                    answer =  ''.join(tokens)
                except:
                    print(idx, answer)
#                     if '人' in answer:
#                         tokens = list(answer)
#                         for x in range(len(tokens)):
#                             if tokens[x] == '人':
#                                 tokens[x - 1] = str(no_ja_dict.get(tokens[x - 1], tokens[x-1]))
                                
#                         answer = ''.join(tokens)
                    
#                     print(idx, answer)
#                     print('------')

            answer_dict[idx] = answer

0it [00:00, ?it/s]

128 が1人います
128 が1人います
------
160 村の入口をくぐる人は1人います
160 村の入口をくぐる人は1人います
------
291 家の中に三人がいます。
291 家の中に3人がいます。
------
732 お店の前に2人座っています
732 お店の前に2人座っています
------
1008 tシャツを買いに来る2人
1008 tシャツを買いに来る2人
------
1010 一人
1010 1人
------
1011 二人
1011 2人
------
1243 女の子は一人で立っています
1243 女の子は1人で立っています
------
1590 赤い服を着た女性が1人います
1590 赤い服を着た女性が1人います
------
1598 一人で行く
1598 1人で行く
------
1604 一人です
1604 1人です
------
1918 電話アクセサリー店には3人の顧客が来ます
1918 電話アクセサリー店には3人の顧客が来ます
------
1966 お店で3人
1966 お店で3人
------
2014 飛行機の下で写真を撮るために立っている1人
2014 飛行機の下で写真を撮るために立っている1人
------
2453 誰も果物狩りに来ませんでした
2453 誰も果物狩りに来ませんでした
------
3074 三人
3074 3人
------
3905 3人
3905 3人
------
3955 出かけた女性が二人います。
3955 出かけた女性が2人います。
------
4240 眼鏡売り場で二人が売っています。
4240 眼鏡売り場で2人が売っています。
------
4887 牛乳屋台に立っている2人がいます
4887 牛乳屋台に立っている2人がいます
------
4902 右隅の壁には6枚の絵がかかっています
4902 右隅の壁には6枚の絵がかかっています
------
4938 一人で
4938 1人で
------


In [20]:
with open('./outputs/public-test/results-how-many-vit-ofa.json', 'w', encoding='utf-8') as f:
    json.dump(answer_dict, f, indent=4, ensure_ascii=False)

## WHAT_COLOR

In [9]:
answer_dict = {}

for i, anno in tqdm(enumerate(test_annotations)):
    question_type = anno['question_type']
    
    if question_type == 'WHAT_COLOR':
        idx = anno['id']
        if anno['language'] == 'en':
            answer = ofa[str(idx)]
#             if len(answer.split()) == 2:
            answer = answer.replace('the ', ' ').strip() 

            answer_dict[idx] = answer
            
with open('./outputs/public-test/results-what_color-ofa.json', 'w', encoding='utf-8') as f:
    json.dump(answer_dict, f, indent=4, ensure_ascii=False)

0it [00:00, ?it/s]